# Train Data - Average and High
Using code from code notebooks 00_1_garmin_gpx.ipynb & 00_2_openweather_api.ipynb to build out the two training datasets for female average performance and high performance. See code notebooks for sources on code. 

---
---

# Imports

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# GPX file
import gpxpy
import gpxpy.gpx

# GPX file
from bs4 import BeautifulSoup
import lxml

# bearing
from geographiclib.geodesic import Geodesic

# distance
import haversine as hs

# API
import requests

%matplotlib inline
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter("ignore")

# Functions
---

## Historical Weather

In [2]:
def hist_weather(unix_datetime):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    
    url_hist_daily = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.752125&lon=-121.288010&dt={unix_datetime}&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
    hist_daily_req = requests.get(url_hist_daily)
    print(f'API Status: {hist_daily_req.status_code}')

    wd_hist = hist_daily_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'uvi', 'clouds', 'visibility', 'wind_speed', 'wind_deg']]

    # Convert dt UNIX to datetime UTC
    wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_hist_df

## GPX to Dataframe

In [23]:
def gpx_to_df(filepath):
    
    # Open .gpx file and parse xml
    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        
    # check length of tracks
    print(f'Tracks: {len(gpx.tracks)}')
    # check number of data points
    print(f'No. of Track Points: {gpx.get_track_points_no()}')
    
    # extract timestamp, lat, lon, and elevation from gpx file
    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'timestamp': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })
    
    # create dataframe
    route_df = pd.DataFrame(route_info)
    
    #######################
    # HEART RATE AND CADENCE
    # add extension data with beautifulsoup
    xml = gpx.to_xml()
    soup = BeautifulSoup(xml, features = 'xml')
    
    # Use Beautiful Soup to find all heart rate and cadence
    hr_soup = soup.find_all('ns3:hr')
    cad_soup = soup.find_all('ns3:cad')
    
    # chop off early points from index - accounts for before cadence sensor calibration
    route_df = route_df[-len(cad_soup):]

    # for loop to extract heart rate and cadence values from each point
    hr_list = []
    cad_list = []
    for i in hr_soup:
        # https://stackoverflow.com/questions/69420686/bs4-element-resultset-elements-to-a-list
        hr_list.append(i.get_text(strip = True))

    for j in cad_soup:
        cad_list.append(j.get_text(strip = True))

    # add columns to dataset from extraction
    route_df['heart_rate'] = hr_list[-len(cad_soup):]
    route_df['cadence'] = cad_list
    
    #######################
    # BEARING
    # get_bearing function
    def get_bearing(lat1, lat2, long1, long2):
        brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
        return brng
    
    # assign bearing in df
    route_df['bearing'] = 0
    for i in range(1, len(route_df)):
        lat1 = route_df.latitude.iloc[i-1]
        lat2 = route_df.latitude.iloc[i]
        long1 = route_df.longitude.iloc[i-1]
        long2 = route_df.longitude.iloc[i]
        bearing = get_bearing(lat1, lat2, long1, long2)
        # https://stats.stackexchange.com/questions/283572/using-iloc-to-set-values
        route_df.bearing.iloc[[i]] = bearing
    
    ########################    
    # ELAPSED TIME - SECONDS
    route_df['timestamp'] = pd.to_datetime(route_df['timestamp'])
    route_df['time_diff_s'] = 0
    for i in range(1, len(route_df)-1):
        t1 = route_df.iloc[i]['timestamp']
        t2 = route_df.iloc[i-1]['timestamp']
        # https://www.geeksforgeeks.org/how-to-set-cell-value-in-pandas-dataframe/
        route_df.at[i, 'time_diff_s'] = (t1 - t2).seconds
    route_df['total_time_s'] = route_df['time_diff_s'].cumsum(skipna = True)
    
    #######################
    # ELEVATION CHANGE - METERS
    route_df['ele_diff'] = 0
    for i in range(1, len(route_df)-1):
        e1 = route_df.iloc[i]['elevation']
        e2 = route_df.iloc[i-1]['elevation']
        route_df.at[i, 'ele_diff'] = (e1 - e2)
    route_df['total_ele_change'] = round(route_df['ele_diff'].cumsum(skipna = True), 4)
    
    #######################
    # DISTANCE - KILOMETERS
    route_df['lat_lon'] = [(lat, lon) for lat, lon in zip(route_df['latitude'], route_df['longitude'])]
    route_df['dist_diff_km'] = 0
    for i in range(1, len(route_df)-1):
        loc1 = route_df.iloc[i]['lat_lon']
        loc2 = route_df.iloc[i-1]['lat_lon']
        route_df.at[i, 'dist_diff_km'] = round(hs.haversine(loc1, loc2), 4) # kilometers
    route_df['total_dist_km'] = route_df['dist_diff_km'].cumsum(skipna = True)
   
    #######################
    route_df.reset_index(drop = True, inplace = True)
    
    return route_df

# Average Cycling Dataset

In [24]:
average = dict()

for x in range(1, 13):
    average[x] = gpx_to_df(f'../data/average/a_{x}.gpx')

Tracks: 1
No. of Track Points: 400
Tracks: 1
No. of Track Points: 3015
Tracks: 1
No. of Track Points: 853
Tracks: 1
No. of Track Points: 1333
Tracks: 1
No. of Track Points: 157
Tracks: 1
No. of Track Points: 2224
Tracks: 1
No. of Track Points: 1364
Tracks: 1
No. of Track Points: 1070
Tracks: 1
No. of Track Points: 739
Tracks: 1
No. of Track Points: 181
Tracks: 1
No. of Track Points: 1453
Tracks: 1
No. of Track Points: 2304


In [33]:
average[2]['lat_lon'][0]

(38.63356165587902, -121.21797876432538)